# Exam 2022/2 - coding

## Task 1

Implement a Bayes filter for the following system:
* An agent moves between cells of a 10x10 grid, initially residing in cell (0,0) - cells are 0-indexed.
* All the cells in even index rows are red, all the cells in odd index rows are green.
* Motion model: an agend accepts movement commands in 4 directions (u - decrease row number, d - increase row number, r - increase column number, l - decrease column number). With probability 0.8 the agent follows the command (unless it would make the agent go outside of the grid, then it stays in place) and with probability 0.2 it remains in its current position.
* Sensor model: the sensor with probability 0.5 returns a correct reading and with probability 0.5 returns a random color from {R, G}.

Your program should accept as input a string consiting of letters {u,d,r,l,R,G} and compute the probability distribution of the agent's position and print it to the standard output (as a numpy array).

## Task 2


Robot has the following URDF (`robot.urdf`):

```xml
<?xml version="1.0" ?>
<robot name="robot">
    <link name="base_link">
        <inertial>
            <mass value="0" />
            <inertia ixx = "0" ixy = "0" ixz = "0"
                iyx = "0" iyy = "0" iyz = "0"
                izx = "0" izy = "0" izz = "0" />
        </inertial>
    </link>
    <joint name="center_z" type="prismatic">
        <parent link="base_link"/>
        <child link="y_control"/>
        <axis xyz="0 0 1"/>
        <limit effort="1000.0" lower="-0.1" upper="1.1" velocity="0.2"/>
    </joint>
    <link name="y_control">
        <inertial>
            <mass value="0.1" />
            <inertia ixx = "0" ixy = "0" ixz = "0"
                iyx = "0" iyy = "0" iyz = "0"
                izx = "0" izy = "0" izz = "0" />
        </inertial>
    </link>
    <joint name="center_y" type="prismatic">
        <parent link="y_control"/>
        <child link="x_control"/>
        <axis xyz="0 1 0"/>
        <limit effort="1000.0" lower="-1.1" upper="1.1" velocity="0.2"/>
    </joint>
    <link name="x_control">
        <inertial>
            <mass value="0.1" />
            <inertia ixx = "0" ixy = "0" ixz = "0"
                iyx = "0" iyy = "0" iyz = "0"
                izx = "0" izy = "0" izz = "0" />
        </inertial>
    </link>
    <joint name="center_x" type="prismatic">
        <parent link="x_control"/>
        <child link="roll_control"/>
        <axis xyz="1 0 0"/>
        <limit effort="1000.0" lower="-1.1" upper="1" velocity="0.2"/>
    </joint>
    <link name="roll_control">
        <inertial>
            <mass value="0.1" />
            <inertia ixx = "0" ixy = "0" ixz = "0"
                iyx = "0" iyy = "0" iyz = "0"
                izx = "0" izy = "0" izz = "0" />
        </inertial>
    </link>
    <joint name="gripper_roll" type="revolute">
        <parent link="roll_control"/>
        <child link="gripper_link"/>
        <axis xyz="0 0 1"/>
        <limit lower="-31.4" upper="31.4" velocity="3.14" effort="10000"/>
    </joint>
    <link name="gripper_link">
        <inertial>
            <origin rpy="0 0 0" xyz="0 0 0" />
            <mass value="1" />
            <inertia ixx="0" ixy="0" ixz="0" iyy="0" iyz="0" izz="0" />
        </inertial>
        <visual>
            <origin rpy="0 0 0" xyz="0 0 0" />
            <geometry>
                <box size="0.1 0.1 0.1"/>  
            </geometry>
            <material name="a">
                <color rgba="0.356 0.361 0.376 1" />
            </material>
        </visual>
        <collision>
            <origin rpy="0 0 0" xyz="0 0 0" />
            <geometry>
                <box size="0.1 0.1 0.1"/>
            </geometry>
        </collision>
    </link>
</robot>
```

The python program below simulates the robot and two cubes.
You can see exactly how the simulation looks like by changing `p.DIRECT` to `p.GUI`.

The goal of the robot task is to move:
- the first cube to [0, -0.2, 0] position 
- the second cube to [-0.2, 0.58, 0] position


### Solution:
- should contain only modified version of `task_2_2.py` file (don't submit any other files)
- should modify only part of the file between START and END comment
- should not modify any other parts of the file
- should not directly interfere with pybullet functions, variables, ...
- should use `cube_pos(i)` and `simulate()` functions
- will be graded based on:
  - the success rate reported by `test_exam` function (50% points)
  - the quality of the source code (50% points)

`task_2_2.py` file:

```python
# The file contains a simulation of a robot and two cubes
# You can see what exactly is simulated by changing p.DIRECT to p.GUI
# The goal of the robot task is to move:
#   - the first cube to [0, -0.2, 0] position 
#   - the second cube to [-0.2, 0.58, 0] position
#
# Solution:
# - should contain only the move_block() function
# - should not modify any other parts of the file
# - should not directly use pybullet
# - should use cube_pos(i) and simulate() functions
# - will be graded based on the success rate reported by test_exam function


import random

import pybullet as p
import pybullet_data
from scipy.spatial import distance

NUMBER_OF_TESTS = 100
p.connect(p.DIRECT)
p.setAdditionalSearchPath(pybullet_data.getDataPath())


def build_world():
    global robot, obstacle, cube_1, cube_2
    p.resetSimulation()
    p.setGravity(0, 0, -10)
    p.loadURDF("plane.urdf", [0, 0, -0.1], useFixedBase=True)
    robot = p.loadURDF("robot.urdf")
    x = random.uniform(-0.3, 0.3)
    y = random.uniform(0.2, 0.7)
    cube_1 = p.loadURDF("cube.urdf", [x, y, 0.1], globalScaling=0.2)
    cube_2 = p.loadURDF("cube.urdf", [x, y, 0.31], globalScaling=0.2)


def cube_pos(i):
    assert i in [0, 1]
    if i == 0:
        return p.getBasePositionAndOrientation(cube_1)[0]
    if i == 1:
        return p.getBasePositionAndOrientation(cube_2)[0]


def simulate(x, y, z, targetVelocity=0.1, steps=5000):
    p.setJointMotorControl2(robot, 0, p.POSITION_CONTROL, z, maxVelocity=targetVelocity)
    p.setJointMotorControl2(robot, 1, p.POSITION_CONTROL, y, maxVelocity=targetVelocity)
    p.setJointMotorControl2(robot, 2, p.POSITION_CONTROL, x, maxVelocity=targetVelocity)
    for i in range(steps):
        p.stepSimulation()


def move_block():
    # YOUR CODE GOES HERE ...
    # this example achieves 0.03 success rate
    # BEGIN

    pos = cube_pos(1)
    simulate(1, 0, 0)
    simulate(1, pos[1], 0.3)
    simulate(pos[0], pos[1], 0.3)
    pos = cube_pos(0)
    simulate(pos[0], pos[1] + 0.2, 0.3)
    simulate(pos[0], pos[1] + 0.2, 0)
    simulate(pos[0], pos[1] - 0.6, 0)
    # END


def test_exam():
    success = 0
    for seed in range(NUMBER_OF_TESTS):
        random.seed(seed)
        print("test", seed)
        build_world()
        move_block()
        if (
            distance.euclidean(cube_pos(0), [0, -0.2, 0]) < 0.1
            and distance.euclidean(cube_pos(1), [-0.2, 0.58, 0]) < 0.1
        ):
            success += 1
    print("Success rate", success / NUMBER_OF_TESTS)
    return success / NUMBER_OF_TESTS


test_exam()
```